In [139]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Training Accuracy  without GridSerchCV

In [140]:
df = pd.read_csv('https://raw.githubusercontent.com/plenoi/Clinic/master/ultima_all_clean.csv')
df = df.set_index('hn')
df.head()

,age,parity,hiv,menopaus,disease,surgery,conization,OPDsize,appearance,stage,...,nodeyiel,RHlvsi,depth,size,utmet,vgmargin,vgmet,pelvicme,pmmet,adnmet
hn,,,,,,,,,,,,,,,,,,,,,
2631840,52,3,0.0,0.0,1,1,0.0,5.0,4.0,5.0,...,21.0,0.0,3.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0
2633481,32,2,0.0,0.0,0,1,0.0,5.0,1.0,5.0,...,11.0,0.0,3.0,5.0,0.0,0.0,0.0,1.0,0.0,2.0
2634477,52,2,0.0,0.0,0,0,0.0,5.0,NaN,5.0,...,35.0,6.0,3.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0
2633633,38,2,0.0,0.0,0,0,0.0,2.0,1.0,4.0,...,20.0,16.0,3.0,3.8,0.0,0.0,0.0,0.0,0.0,2.0
2630496,55,3,0.0,1.0,0,0,1.0,0.0,5.0,4.0,...,17.0,9.0,3.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [141]:
df_clean_column = df.drop(['appearance','Wardsize','RHlvsi','depth','size','nodeyiel','vgmargin','pelvicme','adnmet'],axis=1) #1 = column
df_clean_column.isnull().sum(axis=0)

age            0
parity         0
hiv            4
menopaus       1
disease        0
surgery        0
conization     5
OPDsize       17
stage         24
pchemo         1
finalhisto    10
utmet         98
vgmet         97
pmmet         94
dtype: int64

In [142]:
df_clean = df_clean_column.dropna(axis=0)
df_clean.isnull().sum(axis=0)

age           0
parity        0
hiv           0
menopaus      0
disease       0
surgery       0
conization    0
OPDsize       0
stage         0
pchemo        0
finalhisto    0
utmet         0
vgmet         0
pmmet         0
dtype: int64

In [143]:
y = df_clean['pmmet'].values 
X = df_clean.drop(['pmmet'],axis = 1).values 

# **METHOD 1: HOLD OUT**

In [144]:
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=0)

In [145]:
from sklearn.model_selection import train_test_split 
X_train, X_val, y_train, y_val = train_test_split(X_train,y_train,test_size=0.2,random_state=0)

In [146]:
from sklearn.preprocessing import MinMaxScaler 
scaler = MinMaxScaler(feature_range=(0, 1)) 
scaler.fit(X_train) 
X_train_norm = scaler.transform(X_train) 
X_val_norm = scaler.transform(X_val) 

In [147]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0, solver='liblinear') 
clf.fit(X_train_norm, y_train) 
yp = clf.predict(X_val_norm)
acc = sum(yp == y_val)/len(y_val)
print("Hold out training accuracy: "+str(acc))

Hold out training accuracy: 0.8174603174603174


# **METHOD 2: CROSS VALIDATION**

In [148]:
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=0)

In [149]:
from sklearn.preprocessing import MinMaxScaler 
scaler = MinMaxScaler(feature_range=(0, 1)) 
scaler.fit(X_train) 
X_train_norm = scaler.transform(X_train) 

In [150]:
from sklearn.model_selection import cross_val_score 
clf = LogisticRegression(random_state=0, solver='liblinear') 
acc = cross_val_score(clf, X_train_norm, y_train, cv=10)
print("10CV(Cross validation): "+str(acc.mean()))

10CV(Cross validation): 0.8137079365079366


ถ้าเป็นวิธี leave one out ก็กำหนด cv เป็นจำนวน data ไปเลย 

In [151]:
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=0)

In [152]:
from sklearn.preprocessing import MinMaxScaler 
scaler = MinMaxScaler(feature_range=(0, 1)) 
scaler.fit(X_train) 
X_train_norm = scaler.transform(X_train) 
X_test_norm = scaler.transform(X_test)

In [153]:
from sklearn.model_selection import cross_val_score 
clf = LogisticRegression(random_state=0, solver='liblinear') 
clf.fit(X_train_norm, y_train)
yp = clf.predict(X_test_norm)
acc = sum(yp == y_test)/len(y_test)
print("Test accuracy: "+str(acc.mean()))

Test accuracy: 0.8761904761904762


# **Constant Feature**

In [154]:
df.var(axis=0) #เช็คความแปรปรวน

age               81.065940
parity             1.795348
hiv                0.006936
menopaus           0.207860
disease            0.215740
surgery            0.234638
conization         0.226518
OPDsize            3.664245
appearance         2.997285
stage              1.383333
pchemo             0.149920
Wardsize           3.361144
finalhisto         0.744274
nodeyiel         114.208070
RHlvsi        231725.557678
depth              0.509143
size               3.207926
utmet              0.153174
vgmargin           0.189784
vgmet              0.357326
pelvicme           0.388821
pmmet              0.146025
adnmet             0.732903
dtype: float64

hiv มีความแปรปรวนต่ำมากๆ แสดงว่าข้อมูลคล้ายกันเยอะมากๆ อาจไม่จำเป็นในการนำมาเป็น feature

# **Duplicate Feature**

In [155]:
df.drop_duplicates() #จะลบข้อมูลที่มีความซ้ำซ้อนกันให้เหลือตัวเดียว

,age,parity,hiv,menopaus,disease,surgery,conization,OPDsize,appearance,stage,...,nodeyiel,RHlvsi,depth,size,utmet,vgmargin,vgmet,pelvicme,pmmet,adnmet
hn,,,,,,,,,,,,,,,,,,,,,
2631840,52,3,0.0,0.0,1,1,0.0,5.0,4.0,5.0,...,21.0,0.0,3.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0
2633481,32,2,0.0,0.0,0,1,0.0,5.0,1.0,5.0,...,11.0,0.0,3.0,5.0,0.0,0.0,0.0,1.0,0.0,2.0
2634477,52,2,0.0,0.0,0,0,0.0,5.0,NaN,5.0,...,35.0,6.0,3.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0
2633633,38,2,0.0,0.0,0,0,0.0,2.0,1.0,4.0,...,20.0,16.0,3.0,3.8,0.0,0.0,0.0,0.0,0.0,2.0
2630496,55,3,0.0,1.0,0,0,1.0,0.0,5.0,4.0,...,17.0,9.0,3.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3169688,47,2,0.0,0.0,0,0,1.0,0.0,5.0,4.0,...,42.0,5.0,2.0,1.1,0.0,0.0,0.0,0.0,0.0,2.0
3167041,39,1,0.0,0.0,0,0,1.0,0.0,5.0,2.0,...,20.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,2.0
2623351,46,2,0.0,0.0,0,0,1.0,0.0,5.0,1.0,...,13.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# **Redundant Feature**

In [156]:
s = df.corr().unstack() #อันไหนค่าใกล้ 1 ถือว่าใกล้เคียงกันมากๆ 
s

age     age         1.000000
        parity      0.271321
        hiv        -0.069633
        menopaus    0.721046
        disease     0.323504
                      ...   
adnmet  vgmargin   -0.032079
        vgmet      -0.027424
        pelvicme    0.035556
        pmmet      -0.073738
        adnmet      1.000000
Length: 529, dtype: float64

In [157]:
so = s.sort_values(ascending=False)
so[0:50]

age         age           1.000000
finalhisto  finalhisto    1.000000
menopaus    menopaus      1.000000
disease     disease       1.000000
surgery     surgery       1.000000
conization  conization    1.000000
OPDsize     OPDsize       1.000000
appearance  appearance    1.000000
stage       stage         1.000000
pchemo      pchemo        1.000000
nodeyiel    nodeyiel      1.000000
parity      parity        1.000000
RHlvsi      RHlvsi        1.000000
depth       depth         1.000000
size        size          1.000000
utmet       utmet         1.000000
vgmargin    vgmargin      1.000000
vgmet       vgmet         1.000000
pelvicme    pelvicme      1.000000
pmmet       pmmet         1.000000
hiv         hiv           1.000000
Wardsize    Wardsize      1.000000
adnmet      adnmet        1.000000
OPDsize     Wardsize      0.875699
Wardsize    OPDsize       0.875699
size        Wardsize      0.804733
Wardsize    size          0.804733
appearance  conization    0.794773
conization  appearan

OPDsize กับ Wardsize  ใกล้เคียงกัน 0.875699

# **Feature ที่ไม่ใช่ Feature (ตามไม่ทัน)**

การเอา label มาเป็น feature ทำให้ค่า accuracy สูงกว่าปกติ (95-100 ไรงี้)

In [158]:
s = df.corr().unstack() #อันไหนค่าใกล้ 1 ถือว่าใกล้เคียงกันมากๆ 
s

age     age         1.000000
        parity      0.271321
        hiv        -0.069633
        menopaus    0.721046
        disease     0.323504
                      ...   
adnmet  vgmargin   -0.032079
        vgmet      -0.027424
        pelvicme    0.035556
        pmmet      -0.073738
        adnmet      1.000000
Length: 529, dtype: float64

In [159]:
so = s.sort_values(ascending=False)
so[0:50]

age         age           1.000000
finalhisto  finalhisto    1.000000
menopaus    menopaus      1.000000
disease     disease       1.000000
surgery     surgery       1.000000
conization  conization    1.000000
OPDsize     OPDsize       1.000000
appearance  appearance    1.000000
stage       stage         1.000000
pchemo      pchemo        1.000000
nodeyiel    nodeyiel      1.000000
parity      parity        1.000000
RHlvsi      RHlvsi        1.000000
depth       depth         1.000000
size        size          1.000000
utmet       utmet         1.000000
vgmargin    vgmargin      1.000000
vgmet       vgmet         1.000000
pelvicme    pelvicme      1.000000
pmmet       pmmet         1.000000
hiv         hiv           1.000000
Wardsize    Wardsize      1.000000
adnmet      adnmet        1.000000
OPDsize     Wardsize      0.875699
Wardsize    OPDsize       0.875699
size        Wardsize      0.804733
Wardsize    size          0.804733
appearance  conization    0.794773
conization  appearan

ถ้า feature ไหนใกล้เคียง  label มากๆ ให้ drop ทิ้ง

# **Feature Reduction** 

การยุบรวม feature เข้าด้วยกัน เป็น feature ใหม่ จากเยอะๆ ให้เหลือตามจำนวนที่เราต้องการ

**Method 1: PCA**

In [160]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 3)
pca.fit(X_train)
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)

In [161]:
from sklearn.preprocessing import MinMaxScaler 
scaler = MinMaxScaler(feature_range=(0, 1)) 
scaler.fit(X_train_pca) 
X_train_norm = scaler.transform(X_train_pca) 
X_test_norm = scaler.transform(X_test_pca)

In [162]:
from sklearn.model_selection import cross_val_score 
clf = LogisticRegression(random_state=0, solver='liblinear') 
clf.fit(X_train_norm, y_train)
yp = clf.predict(X_test_norm)
acc = sum(yp == y_test)/len(y_test)
print("Test accuracy: "+str(acc.mean()))

Test accuracy: 0.8507936507936508


**Method 2: MDS**

**Method 3: t-SNE**

# **Feature selection**

**Method 1: Filter method** 

**Method 2: Wrapper method**

**Method 3: Embedded method**

# **Pipeline**

In [163]:
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
clf = Pipeline([
    ('scaler',MinMaxScaler(feature_range=(0,1))), #normalization
    ('feature_selection',SelectKBest(f_classif, k=5)), #select feature
    ('classification', LogisticRegression(random_state=0, solver='liblinear')) #classifier
])
clf.fit(X_train, y_train)

Pipeline(steps=[('scaler', MinMaxScaler()),
                ('feature_selection', SelectKBest(k=5)),
                ('classification',
                 LogisticRegression(random_state=0, solver='liblinear'))])

In [164]:
from sklearn.model_selection import cross_val_score
acc = cross_val_score(clf, X_train, y_train, cv=10)
print("10CV Traning accuracy: "+str(acc.mean()))

10CV Traning accuracy: 0.8145015873015874


In [165]:
clf.predict(X_test)
acc = sum(yp == y_test)/len(y_test)
print("Test Training accuracy: "+str(acc))

Test Training accuracy: 0.8507936507936508


# **ASSIGNMENT**

Cleansing

In [166]:
df = pd.read_csv('https://raw.githubusercontent.com/plenoi/Clinic/master/ultima_all_clean.csv')
df = df.set_index('hn')
df_clean_column = df.drop(['appearance','Wardsize','RHlvsi','depth','size','nodeyiel','vgmargin','pelvicme','adnmet'],axis=1) #1 = column
df_clean = df_clean_column.dropna(axis=0)
y = df_clean['pmmet'].values 
X = df_clean.drop(['pmmet'],axis = 1).values 
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=0)

In [167]:
np.unique(y)

array([0., 1.])

In [168]:
np.count_nonzero(y==0)

1291

In [169]:
np.count_nonzero(y==1)

280

Preprocessing

In [170]:
from imblearn.under_sampling import RandomUnderSampler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.svm import SVC

undersample = RandomUnderSampler(sampling_strategy='majority') #resample of imbalanced label
X_res, y_res = undersample.fit_resample(X_train, y_train)

from sklearn.pipeline import Pipeline
clf = Pipeline([
    ('scaler',MinMaxScaler(feature_range=(0,1))), #normalization
    ('feature_selection',SelectKBest(f_classif, k=5)), #select feature
    ('classification',SVC()) #classifier
])
clf.fit(X_res, y_res)

Pipeline(steps=[('scaler', MinMaxScaler()),
                ('feature_selection', SelectKBest(k=5)),
                ('classification', SVC())])

Trainning

In [184]:
X_train.shape

(1256, 13)

In [218]:
from sklearn.model_selection import cross_val_score
acc = cross_val_score(clf, X_train, y_train, cv=500) #cross_validation (1023)
print("10CV Traning accuracy: "+str(acc.mean()*100))

/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_split.py:672: UserWarning: The least populated class in y has only 233 members, which is less than n_splits=500.
  % (min_groups, self.n_splits)), UserWarning)


10CV Traning accuracy: 84.36666666666666


Testing

In [219]:
clf.predict(X_test)
acc = sum(yp == y_test)/len(y_test)
print("Test Training accuracy: "+str(acc*100))

Test Training accuracy: 85.07936507936508
